<a href="https://colab.research.google.com/github/usmanqadeer1/PSGD/blob/master/PSGD_MNIST_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SET UP

In [2]:
# !kill -9 -1
import tensorflow as tf
# tf.test.gpu_device_name()
print(tf.__version__)
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import numpy as np
import time
import math

from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
%load_ext tensorboard

2.2.0-rc3
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from google.colab import drive
from google.colab import files
import sys

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/Preconditioned SGD/MNIST/'
results_dir = base_dir + 'results1/'
logs_dir = base_dir + 'log'
sys.path.append(base_dir)
import preconditioned_stochastic_gradient_descent as psgd 

Mounted at /content/gdrive/


#Defining Functions

In [0]:
def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
  fig.show()

def plot_acc_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis=dict(range=[0.95, 1]))
  fig.show()


# Set up for MNIST classification CNN

## Download MNIST Data

In [0]:
"""MNIST, classification"""
np.random.seed(0)

# Parameter Settings
batch_size = 128
num_f = 32  # number of features 
ITERATIONS = 10000
GAP = 100

dtype = tf.float32

mnist = tf.keras.datasets.mnist.load_data()

In [7]:
(mnist_train_data, mnist_train_labels), (mnist_test_data, mnist_test_labels) = mnist

print('total images:',mnist_train_data.shape[0])
print('training images:',mnist_train_data.shape[0])
print('test images:',mnist_test_data.shape[0])
print('number of features:',mnist_train_data.shape[1])


# reshape data as 28x28x1 image 
train_data = np.reshape(2.0*mnist_train_data - 1.0, [60000, 28, 28, 1])

#binarize the labels
train_label = tf.keras.utils.to_categorical(mnist_train_labels)
test_label = tf.keras.utils.to_categorical(mnist_test_labels)

total images: 60000
training images: 60000
test images: 10000
number of features: 28


In [0]:
def get_batches():
    # generate 60000 numbers randomly without replacement
    rp = np.random.permutation(train_data.shape[0])
    x = -np.ones([batch_size, 32, 32, 1])
    
    # augumentation: randomly shifting image by +-2 pixels 
    for i in range(batch_size):
        m = math.floor(5.0*np.random.rand())
        n = math.floor(5.0*np.random.rand())
        x[i, m:m+28, n:n+28] = train_data[rp[i]]
    y = train_label[rp[0:batch_size]]
    return x, y

## Defining CNN model for training.
The model consists of 2 convolutional layers, 1 avgpool layer, 2 convolutional layers, 1 avg pool layer, 1 FC layer, 1 FC layer.

In [0]:
train_inputs = tf.placeholder(dtype, [batch_size, 32, 32, 1])
train_outputs = tf.placeholder(dtype, [batch_size, 10])
adam_step = tf.placeholder(tf.float32, shape = ())

In [0]:
# (hight, width, in_ch, out_ch) tensor --> (hight * width * in_ch, out_ch) matrix 
W1 = tf.Variable(np.random.normal(loc=0.0, scale=1.0/np.sqrt(3*3*1+1), size=[3*3*1+1, num_f]), dtype=dtype, name = 'W1')
W2 = tf.Variable(np.random.normal(loc=0.0, scale=1.0/np.sqrt(3*3*num_f+1), size=[3*3*num_f+1, num_f]), dtype=dtype, name = 'W2')
W3 = tf.Variable(np.random.normal(loc=0.0, scale=1.0/np.sqrt(3*3*num_f+1), size=[3*3*num_f+1, num_f]), dtype=dtype, name = 'W3')
W4 = tf.Variable(np.random.normal(loc=0.0, scale=1.0/np.sqrt(3*3*num_f+1), size=[3*3*num_f+1, num_f]), dtype=dtype, name = 'W4')
W5 = tf.Variable(np.random.normal(loc=0.0, scale=1.0/np.sqrt(5*5*num_f+1), size=[5*5*num_f+1, 10]), dtype=dtype, name = 'W5')
Ws = [W1, W2, W3, W4, W5]

# we use the traditional tanh
def model(Ws, inputs):
    
    W1, W2, W3, W4, W5 = Ws
    w1 = tf.reshape(W1[:-1], [3, 3, 1, num_f])
    b1 = W1[-1]
    x1 = tf.tanh( tf.nn.conv2d(input = inputs,filters = w1, strides = [1,1,1,1], padding = 'VALID', name = 'Conv_1') + b1 )
    
        
    w2 = tf.reshape(W2[:-1], [3, 3, num_f, num_f])
    b2 = W2[-1]
    x2 = tf.tanh( tf.nn.conv2d(input = x1, filters = w2, strides = [1,1,1,1], padding = 'VALID', name = 'Conv_2') + b2 )
    
    x2 = tf.nn.avg_pool(value = x2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name = 'Avg_pool_2')
        
    w3 = tf.reshape(W3[:-1], [3, 3, num_f, num_f])
    b3 = W3[-1]
    x3 = tf.tanh( tf.nn.conv2d(x2, w3, [1,1,1,1], 'VALID',name = 'Conv3') + b3 )
        
    w4 = tf.reshape(W4[:-1], [3, 3, num_f, num_f])
    b4 = W4[-1]
    x4 = tf.tanh( tf.nn.conv2d(x3, w4, [1,1,1,1], 'VALID') + b4, name = 'Conv4' )
    
    x4 = tf.nn.avg_pool(value = x4, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID', name = 'Avg_pool4')
    
    batch_size = inputs.shape.as_list()[0]
    x4_flat = tf.reshape(x4, [batch_size, -1], name = 'flatten')
    ones = tf.ones([batch_size, 1], dtype=dtype)
    y = tf.matmul(tf.concat([x4_flat, ones], axis = 1), W5,name = 'fc')
    tf.summary.histogram("preds", y)
    return y

test_data = -np.ones([10000, 32, 32, 1])
test_data[:,2:30,2:30] = np.reshape(2.0*mnist_test_data - 1.0, [10000, 28, 28, 1])

## Define cross entropy as Training Loss function (Loss function) and Test Loss function

In [0]:
def train_criterion(Ws):
    y = model(Ws, train_inputs)
    train_accuracy =tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_outputs, axis = 1), tf.argmax(y, axis = 1)), dtype = dtype))
    train_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=train_outputs, logits=y))
    return train_loss, train_accuracy

# classification error rate
def test_criterion(Ws):
    y = model(Ws, tf.constant(test_data, dtype=dtype))
    test_accuracy =tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_label, axis = 1), tf.argmax(y, axis = 1)), dtype = dtype))
    test_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=test_label, logits=y))
    return test_loss, test_accuracy

In [0]:
test_outputs = mnist_test_labels

def final_outputs(Ws):
  y = model(Ws, tf.constant(test_data, dtype=dtype))
  test_preds = [tf.argmax(y[i]) for i in range(10000)]
  return test_preds


# First Order Methods

## SGD

In [47]:
step_size = 0.05
grad_norm_clip_thr = 1e0 # may need gradients cliping for RNN training; otherwise, set it to an extremely large value  


with tf.Session() as sess:
  
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    test_loss, test_accuracy = test_criterion(Ws)


    sess.run(tf.global_variables_initializer())

    # writer = tf.summary.FileWriter(base_dir+'1')
    # writer.add_graph(sess.graph)  
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()

    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
     
        t0 = time.time()
        # merge = tf.summary.merge_all()
        # _train_loss, _train_accuracy, myWs, summary = sess.run([train_loss, train_accuracy, update_Ws, merge],
        #                           {train_inputs: _train_inputs, train_outputs: _train_outputs})
        _train_loss, _train_accuracy, myWs = sess.run([train_loss, train_accuracy, update_Ws],
                                  {train_inputs: _train_inputs, train_outputs: _train_outputs})   
        # writer.add_summary(summary)
        Time.append(time.time() - t0)
        
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)

            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)
# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))
 
scipy.io.savemat(results_dir + 'SGD.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.4049949645996094; test loss: 2.325472831726074; train accuracy: 0.078125; test accuracy: 0.0843999981880188; Time: 6.358717679977417
100: train loss: 1.2767315725445745; test loss: 0.5244912505149841; train accuracy: 0.6102406249999998; test accuracy: 0.8628000020980835; Time: 6.869877576828003
200: train loss: 0.903284734816955; test loss: 0.2953112721443176; train accuracy: 0.7286362400635681; test accuracy: 0.9226999878883362; Time: 7.373209476470947
300: train loss: 0.596813684978206; test loss: 0.19326435029506683; train accuracy: 0.8234783918416219; test accuracy: 0.9491999745368958; Time: 7.8747718334198
400: train loss: 0.42241290693218636; test loss: 0.16318072378635406; train accuracy: 0.8759638387545914; test accuracy: 0.9537000060081482; Time: 8.37878680229187
500: train loss: 0.3282435389505719; test loss: 0.13811151683330536; train accuracy: 0.9031870348839306; test accuracy: 0.960099995136261; Time: 8.884768724441528
600: train loss: 0.27658525001029566;

## ADAM

In [48]:
# Adam 
step_size = 0.001
beta1 = 0.9
beta2 = 0.999
offset = 1e-9
         
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter( './logs/1/train ', sess.graph)
    grads_vars = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]
    grads_moment = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    new_grads_moment = [beta1*m + (1.0 - beta1)*g for (m, g) in zip(grads_moment, grads)]
    new_grads_vars = [beta2*v + (1.0 - beta2)*g*g for (v, g) in zip(grads_vars, grads)]
    new_grads_moment_hat = [m/(1-beta1**adam_step) for m in new_grads_moment]
    new_grads_vars_hat = [v/(1-beta2**adam_step) for v in new_grads_vars]
          
    new_Ws = [W - step_size*m/(tf.sqrt(v) + offset) for (W, m, v) in zip(Ws, new_grads_moment_hat, new_grads_vars_hat)]
  
    update_Ws = [tf.assign(old, new) for (old, new) in zip(Ws, new_Ws)]
    update_grads_vars = [tf.assign(old, new) for (old, new) in zip(grads_vars, new_grads_vars)]
    update_grads_moment = [tf.assign(old, new) for (old, new) in zip(grads_moment, new_grads_moment)]
    
    test_loss, test_accuracy = test_criterion(Ws)

    merge = tf.summary.merge_all()
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    t = 0
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches()
     
        t = t + 1
        t0 = time.time()
        _train_loss,_train_accuracy ,_,_,_ = sess.run([train_loss, train_accuracy, update_Ws, update_grads_vars, update_grads_moment],
                                      {train_inputs: _train_inputs, train_outputs: _train_outputs, adam_step:t})  
        
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))

scipy.io.savemat(results_dir +'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3887710571289062; test loss: 2.22733211517334; train accuracy: 0.0703125; test accuracy: 0.2168000042438507; Time: 6.963823080062866
100: train loss: 0.8725893724679947; test loss: 0.21460556983947754; train accuracy: 0.7312921874999996; test accuracy: 0.9394999742507935; Time: 7.512877941131592
200: train loss: 0.4970981447247418; test loss: 0.10592687875032425; train accuracy: 0.8495780837615549; test accuracy: 0.9703999757766724; Time: 8.060205459594727
300: train loss: 0.30328636259727254; test loss: 0.0897936299443245; train accuracy: 0.9093921919939223; test accuracy: 0.9753999710083008; Time: 8.61012887954712
400: train loss: 0.20816645816731053; test loss: 0.060148052871227264; train accuracy: 0.9390854232239534; test accuracy: 0.9836999773979187; Time: 9.156958103179932
500: train loss: 0.15451323969475314; test loss: 0.051190536469221115; train accuracy: 0.9552806648934129; test accuracy: 0.9860000014305115; Time: 9.704942464828491
600: train loss: 0.12525816

## RMSprop

In [49]:
# RMSProp  
step_size = 0.0005
max_mu = 0.99
offset = 1e-9
         
with tf.Session() as sess:
    grads_vars = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]
    mu = tf.Variable(initial_value=0.0, trainable=False, dtype=dtype) # forgetting factor for grads**2 estimation 
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    new_grads_vars = [mu*old + (1.0 - mu)*new*new for (old, new) in zip(grads_vars, grads)]
    new_Ws = [W - step_size*g/tf.sqrt(v + offset) for (W, g, v) in zip(Ws, grads, new_grads_vars)]
    new_mu = tf.minimum(max_mu, 1.0/(2.0 - mu))
    
    update_Ws = [tf.assign(old, new) for (old, new) in zip(Ws, new_Ws)]
    update_grads_vars = [tf.assign(old, new) for (old, new) in zip(grads_vars, new_grads_vars)]
    update_mu = tf.assign(mu, new_mu)
    
    test_loss, test_accuracy = test_criterion(Ws)
      
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
     
        t0 = time.time()
        _train_loss,_train_accuracy ,_,_,_ = sess.run([train_loss, train_accuracy, update_Ws, update_grads_vars, update_mu],
                                      {train_inputs: _train_inputs, train_outputs: _train_outputs})  
        Time.append(time.time() - t0)
        
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
    t = final_outputs(Ws)
    test_predictions = sess.run(t)


# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))

  
scipy.io.savemat(results_dir + 'rmsprop.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3804333209991455; test loss: 2.2963647842407227; train accuracy: 0.078125; test accuracy: 0.09830000251531601; Time: 7.600159168243408
100: train loss: 1.1176577069401739; test loss: 0.34561532735824585; train accuracy: 0.6602835937499998; test accuracy: 0.9190999865531921; Time: 8.12697172164917
200: train loss: 0.6773411398583398; test loss: 0.17400285601615906; train accuracy: 0.8002087965775432; test accuracy: 0.954800009727478; Time: 8.654669284820557
300: train loss: 0.4334770073498253; test loss: 0.13219572603702545; train accuracy: 0.8751217370936799; test accuracy: 0.9635000228881836; Time: 9.17665958404541
400: train loss: 0.30815603959024274; test loss: 0.095955029129982; train accuracy: 0.911550494360015; test accuracy: 0.9743000268936157; Time: 9.701686143875122
500: train loss: 0.2322795113560987; test loss: 0.09619250893592834; train accuracy: 0.9332260206017013; test accuracy: 0.972599983215332; Time: 10.223303079605103
600: train loss: 0.18690569624221

# Preconditioning Methods

## KRON

In [50]:
# PSGD with Kronecker Product Preconditioning
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 
    
    Qs_left = [tf.Variable(tf.eye(row, dtype=dtype), trainable=False) for row in rows]
    Qs_right = [tf.Variable(tf.eye(col, dtype=dtype), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]

    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    delta_grads = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, delta_grads)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_approx, qs_r_kron_approx = sess.run([Qs_left, Qs_right])
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'kron_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.352512836456299; test loss: 2.29834246635437; train accuracy: 0.0546875; test accuracy: 0.1054999977350235; Time: 8.061787843704224
100: train loss: 0.9352476075857882; test loss: 0.18265344202518463; train accuracy: 0.7125828125; test accuracy: 0.9513999819755554; Time: 9.574213027954102
200: train loss: 0.48318386733946594; test loss: 0.08360634744167328; train accuracy: 0.8557823398936957; test accuracy: 0.977400004863739; Time: 11.074598550796509
300: train loss: 0.261775392530891; test loss: 0.05746502801775932; train accuracy: 0.922872444180076; test accuracy: 0.9836000204086304; Time: 12.57418942451477
400: train loss: 0.1607586518939462; test loss: 0.043203771114349365; train accuracy: 0.9538522777390354; test accuracy: 0.9887999892234802; Time: 14.072478771209717
500: train loss: 0.109429627991525; test loss: 0.039045240730047226; train accuracy: 0.9691152518754109; test accuracy: 0.9882000088691711; Time: 15.571375131607056
600: train loss: 0.0839520889103970

## SCAN

In [51]:
# PSGD with SCAW Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   
  
    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 

    qs_left = [tf.Variable(tf.concat([tf.ones((1,row)), tf.zeros((1, row))], axis=0), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.ones((1, col)), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_scan(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_qs = [psgd.update_precond_scan(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'scan_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3422911167144775; test loss: 2.293926954269409; train accuracy: 0.1328125; test accuracy: 0.13989999890327454; Time: 8.682962417602539
100: train loss: 0.9978855101197953; test loss: 0.22634975612163544; train accuracy: 0.6891453124999997; test accuracy: 0.9365000128746033; Time: 10.027372360229492
200: train loss: 0.5514038897665152; test loss: 0.10817119479179382; train accuracy: 0.8284217718227347; test accuracy: 0.9678999781608582; Time: 11.363677740097046
300: train loss: 0.31653387096998886; test loss: 0.06137393042445183; train accuracy: 0.8999262419215982; test accuracy: 0.9811999797821045; Time: 12.697064399719238
400: train loss: 0.1957667648789433; test loss: 0.08449813723564148; train accuracy: 0.9380837742543915; test accuracy: 0.974399983882904; Time: 14.033806324005127
500: train loss: 0.14027595180281924; test loss: 0.053428057581186295; train accuracy: 0.9563551888310258; test accuracy: 0.983299970626831; Time: 15.369342565536499
600: train loss: 0.114

## SCAW

In [52]:
# PSGD with SCAW Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   

    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    qs_right =  [tf.Variable(tf.ones(shape = (1, W.shape.as_list()[1]), dtype=dtype), trainable=False) for W in Ws]
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_scaw(ql, qr, g) for (ql, qr, g) in zip(Qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_scaw(ql, qr, dw, dx) for (ql, qr, dw, dx) in zip(Qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(ql, new_q[0]), tf.assign(qr, new_q[1])] for (ql, qr, new_q) in zip(Qs_left, qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'scaw_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.406951665878296; test loss: 2.280700445175171; train accuracy: 0.0390625; test accuracy: 0.13850000500679016; Time: 9.061846256256104
100: train loss: 0.9543156398713591; test loss: 0.1850489228963852; train accuracy: 0.7070843749999999; test accuracy: 0.9516000151634216; Time: 10.525571823120117
200: train loss: 0.5216721737273958; test loss: 0.07960118353366852; train accuracy: 0.8405636188079111; test accuracy: 0.9754999876022339; Time: 11.983959913253784
300: train loss: 0.3033535008732886; test loss: 0.07525373995304108; train accuracy: 0.9071256938884742; test accuracy: 0.9758999943733215; Time: 13.44930100440979
400: train loss: 0.18942805916044772; test loss: 0.07641396671533585; train accuracy: 0.941123154249782; test accuracy: 0.9767000079154968; Time: 14.921849250793457
500: train loss: 0.1350907170476837; test loss: 0.053063634783029556; train accuracy: 0.9587841912442171; test accuracy: 0.9825000166893005; Time: 16.42529821395874
600: train loss: 0.1089586

## Reverse SCAN

In [53]:
# PSGD with Reverse SCAN Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:  
    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 

    qs_left = [tf.Variable(tf.ones((row, 1)), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.concat([tf.ones((1, col)), tf.zeros((1,col))], axis=0), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_reverse_scan(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_qs = [psgd.update_precond_reverse_scan(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'reverse_scan_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3989572525024414; test loss: 2.274468421936035; train accuracy: 0.03125; test accuracy: 0.11259999871253967; Time: 9.966453313827515
100: train loss: 1.0567421861797572; test loss: 0.2626641094684601; train accuracy: 0.66702265625; test accuracy: 0.9283999800682068; Time: 11.369228839874268
200: train loss: 0.6132335827668665; test loss: 0.15888215601444244; train accuracy: 0.8075778018368527; test accuracy: 0.949400007724762; Time: 12.764479875564575
300: train loss: 0.36061918752643624; test loss: 0.083712138235569; train accuracy: 0.8852634732935418; test accuracy: 0.9742000102996826; Time: 14.160664081573486
400: train loss: 0.2309555365214612; test loss: 0.053850773721933365; train accuracy: 0.927263204477854; test accuracy: 0.9836000204086304; Time: 15.556456804275513
500: train loss: 0.17147340771280872; test loss: 0.07865551859140396; train accuracy: 0.9464382924947589; test accuracy: 0.9754999876022339; Time: 16.94869875907898
600: train loss: 0.13591962183849

##Reverse SCAW 

In [54]:
# PSGD with Reverse SCAW Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   
    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 
    
    qs_left =  [tf.Variable(tf.ones((row,1), dtype=dtype), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.eye(col, dtype=dtype), trainable=False) for col in cols]

    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_reverse_scaw(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_reverse_scaw(ql, qr, dw, dx) for (ql, qr, dw, dx) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(ql, new_q[0]), tf.assign(qr, new_q[1])] for (ql, qr, new_q) in zip(qs_left, qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'reverse_scaw_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.39605712890625; test loss: 2.278923273086548; train accuracy: 0.0703125; test accuracy: 0.13449999690055847; Time: 10.48609709739685
100: train loss: 0.9916962894797321; test loss: 0.2340300977230072; train accuracy: 0.6934687500000001; test accuracy: 0.9384999871253967; Time: 11.836423397064209
200: train loss: 0.5527473058086424; test loss: 0.10474731773138046; train accuracy: 0.8290452164736322; test accuracy: 0.972100019454956; Time: 13.162334680557251
300: train loss: 0.31719461523478604; test loss: 0.08558320254087448; train accuracy: 0.9029190762179533; test accuracy: 0.9742000102996826; Time: 14.490496397018433
400: train loss: 0.21467854254808244; test loss: 0.07513278722763062; train accuracy: 0.933296175570173; test accuracy: 0.9771000146865845; Time: 15.817685842514038
500: train loss: 0.15205770964737178; test loss: 0.051178913563489914; train accuracy: 0.9519744119199359; test accuracy: 0.9843999743461609; Time: 17.143138885498047
600: train loss: 0.12515

## SCAN Pro

In [55]:
# PSGD with SCAN Pro Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   

    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 

    qs_left = [tf.Variable(tf.concat([tf.ones((1, row)), tf.zeros((1,row))], axis=0), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.concat([tf.ones((1, col)), tf.zeros((1,col))], axis=0), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_scan_pro(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_qs = [psgd.update_precond_scan_pro(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'scan_pro_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3781280517578125; test loss: 2.2839367389678955; train accuracy: 0.0859375; test accuracy: 0.11969999969005585; Time: 11.206511974334717
100: train loss: 1.0221012464970338; test loss: 0.24603205919265747; train accuracy: 0.6810210937499998; test accuracy: 0.9283999800682068; Time: 12.70463752746582
200: train loss: 0.5920223488680645; test loss: 0.10427120327949524; train accuracy: 0.81553481076288; test accuracy: 0.9696999788284302; Time: 14.205026865005493
300: train loss: 0.35963396321214314; test loss: 0.10865043848752975; train accuracy: 0.8890100693742983; test accuracy: 0.9664000272750854; Time: 15.707662582397461
400: train loss: 0.23657467427487283; test loss: 0.07656712830066681; train accuracy: 0.9273975119306718; test accuracy: 0.9753999710083008; Time: 17.185430765151978
500: train loss: 0.17364982834987713; test loss: 0.046841174364089966; train accuracy: 0.9446162953189742; test accuracy: 0.9846000075340271; Time: 18.681775331497192
600: train loss: 0.1

## Hybrid SKron

In [13]:
# PSGD with Hybrid SKron Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   

    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 

    qs_left = [tf.Variable(tf.concat([tf.ones((1, row)), tf.zeros((1, row))], axis=0), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.eye(col, dtype=dtype), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_hybrid_skron(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_qs = [psgd.update_precond_hybrid_skron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'hybrid_skron_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.4049949645996094; test loss: 2.2869622707366943; train accuracy: 0.078125; test accuracy: 0.10740000009536743; Time: 9.226301193237305
100: train loss: 0.9907160532206298; test loss: 0.2323305904865265; train accuracy: 0.6882906249999996; test accuracy: 0.9333000183105469; Time: 10.656553268432617
200: train loss: 0.5496377775099641; test loss: 0.15136972069740295; train accuracy: 0.8275272847538304; test accuracy: 0.9538999795913696; Time: 12.089669227600098
300: train loss: 0.3100779465309759; test loss: 0.08349881321191788; train accuracy: 0.9034438194162736; test accuracy: 0.9739000201225281; Time: 13.519777059555054
400: train loss: 0.19530028085419654; test loss: 0.057364124804735184; train accuracy: 0.9402353645911024; test accuracy: 0.9818000197410583; Time: 14.948010444641113
500: train loss: 0.14003243121027822; test loss: 0.06593816727399826; train accuracy: 0.9572900237605413; test accuracy: 0.9782999753952026; Time: 16.378555297851562
600: train loss: 0.11

##Hybrid KScan

In [14]:
# PSGD with Hybrid KScan Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   

    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 

    qs_left = [tf.Variable(tf.eye(row, dtype=dtype), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.concat([tf.ones((1, col)), tf.zeros((1, col))], axis=0), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_hybrid_kscan(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_qs = [psgd.update_precond_hybrid_kscan(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'hybrid_kscan_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3887710571289062; test loss: 2.2904887199401855; train accuracy: 0.0703125; test accuracy: 0.13860000669956207; Time: 0.860135555267334
100: train loss: 0.9845488595396279; test loss: 0.2089725136756897; train accuracy: 0.7054617187499996; test accuracy: 0.9459999799728394; Time: 2.48234224319458
200: train loss: 0.5171744149952544; test loss: 0.09064295142889023; train accuracy: 0.8484288838832584; test accuracy: 0.9763000011444092; Time: 4.102585315704346
300: train loss: 0.28651777161451164; test loss: 0.06740223616361618; train accuracy: 0.917428455211132; test accuracy: 0.9830999970436096; Time: 5.714547395706177
400: train loss: 0.17853573998640934; test loss: 0.04883049800992012; train accuracy: 0.9493884733203084; test accuracy: 0.9865999817848206; Time: 7.329389810562134
500: train loss: 0.12344305702969546; test loss: 0.043190523982048035; train accuracy: 0.9653251618815094; test accuracy: 0.9872000217437744; Time: 8.942263126373291
600: train loss: 0.0960531

##KDiag

In [15]:
# PSGD with KDiag Preconditioning 
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   

    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 

    qs_left =  [tf.Variable(tf.ones((row,1), dtype=dtype), trainable=False) for row in rows]
    qs_right = [tf.Variable(tf.ones((1, col), dtype=dtype), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_kdiag(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_qs = [psgd.update_precond_kdiag(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'kdiag_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3804333209991455; test loss: 2.277445077896118; train accuracy: 0.078125; test accuracy: 0.11339999735355377; Time: 1.1713142395019531
100: train loss: 1.023000215509535; test loss: 0.2541208863258362; train accuracy: 0.6726671874999998; test accuracy: 0.9232000112533569; Time: 2.476010322570801
200: train loss: 0.5793444622594308; test loss: 0.12295564264059067; train accuracy: 0.816227766414133; test accuracy: 0.9610999822616577; Time: 3.7732372283935547
300: train loss: 0.3421442895983909; test loss: 0.07866337895393372; train accuracy: 0.8924765817537257; test accuracy: 0.9750999808311462; Time: 5.068522930145264
400: train loss: 0.2248196931149301; test loss: 0.0704597756266594; train accuracy: 0.9308052099861354; test accuracy: 0.9753999710083008; Time: 6.355468034744263
500: train loss: 0.16590465741409413; test loss: 0.054392460733652115; train accuracy: 0.9493403262563517; test accuracy: 0.9839000105857849; Time: 7.647140979766846
600: train loss: 0.1294015419

## KRON KRON

In [16]:
# PSGD with Kronecker Product Preconditioning
step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    rows = [W.shape.as_list()[0] for W in Ws]
    cols = [W.shape.as_list()[1] for W in Ws] 
    
    Qs_left = [tf.Variable(tf.eye(row, dtype=dtype), trainable=False) for row in rows]
    Qs_right = [tf.Variable(tf.eye(col, dtype=dtype), trainable=False) for col in cols]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)
    
    precond_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, grads)]
    precond_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, precond_grads)]

    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    new_Ws = [W - (step_size_adjust*step_size)*g for (W, g) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]

    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    delta_grads = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, delta_grads)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
    
        t0 = time.time()
        _train_loss, _train_accuracy, _, _ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_approx, qs_r_kron_approx = sess.run([Qs_left, Qs_right])
    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))


scipy.io.savemat(results_dir + 'kron_kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.352512836456299; test loss: 2.29834246635437; train accuracy: 0.0546875; test accuracy: 0.1054999977350235; Time: 1.598461389541626
100: train loss: 0.7497566517174246; test loss: 0.08409646898508072; train accuracy: 0.7698203125000002; test accuracy: 0.975600004196167; Time: 3.179511070251465
200: train loss: 0.34829148347527183; test loss: 0.04525052011013031; train accuracy: 0.8942035265331466; test accuracy: 0.9878000020980835; Time: 4.745014667510986
300: train loss: 0.17308768155725332; test loss: 0.03595742583274841; train accuracy: 0.9481247616003815; test accuracy: 0.9890000224113464; Time: 6.313241958618164
400: train loss: 0.09967959506637074; test loss: 0.028861716389656067; train accuracy: 0.9704332806861677; test accuracy: 0.9912999868392944; Time: 7.882060766220093
500: train loss: 0.06535659670327855; test loss: 0.02634241245687008; train accuracy: 0.981119344067093; test accuracy: 0.9915000200271606; Time: 9.453634977340698
600: train loss: 0.049316699

# ADAM KRON

##  ADAM --> Kronecker Product

In [17]:
# apply adam on grads then computing preconditioner from the adam_grads
# grads ---> adam_grads ---> precond_grads

step_size = 0.02
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   
beta1 = 0.9
beta2 = 0.999
offset = 1e-9

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    grads_vars = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]
    grads_moment = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]

    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    Qs_right = [tf.Variable(tf.eye(W.shape.as_list()[1], dtype=dtype), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)

    new_grads_moment = [beta1*old + (1.0 - beta1)*new for (old, new) in zip(grads_moment, grads)]
    new_grads_vars = [beta2*old + (1.0 - beta2)*new*new for (old, new) in zip(grads_vars, grads)]
    new_grads_moment_hat = [m/(1-beta1**adam_step) for m in new_grads_moment]
    new_grads_vars_hat = [v/(1-beta2**adam_step) for v in new_grads_vars]

    adam_grads = [m/tf.sqrt(v + offset) for (m,v) in zip(new_grads_moment_hat,new_grads_vars_hat)]

    precond_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, adam_grads)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    
    new_Ws = [W - (step_size_adjust*step_size)*pG for (W, pG) in zip(Ws,precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    update_grads_vars = [tf.assign(old, new) for (old, new) in zip(grads_vars, new_grads_vars)]
    update_grads_moment = [tf.assign(old, new) for (old, new) in zip(grads_moment, new_grads_moment)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)     
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    t = 0
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
        t = t + 1
        t0 = time.time()
        _train_loss, _train_accuracy, _, _,_,_ = sess.run([train_loss, train_accuracy, update_Ws, update_grads_moment, update_grads_vars, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs, adam_step: t})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_adam, qs_r_kron_adam = sess.run([Qs_left, Qs_right])

    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))

  
scipy.io.savemat(results_dir + 'adam_kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3422911167144775; test loss: 2.2271106243133545; train accuracy: 0.1328125; test accuracy: 0.2410999983549118; Time: 2.1239004135131836
100: train loss: 0.5537339011058204; test loss: 0.07450881600379944; train accuracy: 0.8259718749999998; test accuracy: 0.9753999710083008; Time: 3.7569997310638428
200: train loss: 0.25980328081247483; test loss: 0.03505900129675865; train accuracy: 0.9186728491064133; test accuracy: 0.9883999824523926; Time: 5.36476469039917
300: train loss: 0.1368506157169717; test loss: 0.029348034411668777; train accuracy: 0.9576611711868063; test accuracy: 0.9912999868392944; Time: 6.960961103439331
400: train loss: 0.0810309802723222; test loss: 0.03540801256895065; train accuracy: 0.9748706050655941; test accuracy: 0.9886999726295471; Time: 8.582138538360596
500: train loss: 0.054930329053497574; test loss: 0.028640570119023323; train accuracy: 0.9827787075883204; test accuracy: 0.9900000095367432; Time: 10.199953079223633
600: train loss: 0.04

##Momentum+Kron

In [18]:
# apply adam on grads then computing preconditioner from the adam_grads
# grads ---> adam_grads ---> precond_grads

step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   
beta1 = 0.9
beta2 = 0.999
offset = 1e-9

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    grads_moment = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]

    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    Qs_right = [tf.Variable(tf.eye(W.shape.as_list()[1], dtype=dtype), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)

    new_grads_moment = [beta1*old + (1.0 - beta1)*new for (old, new) in zip(grads_moment, grads)]

    precond_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, new_grads_moment)]
    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    
    new_Ws = [W - (step_size_adjust*step_size)*pG for (W, pG) in zip(Ws,precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    update_grads_moment = [tf.assign(old, new) for (old, new) in zip(grads_moment, new_grads_moment)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)     
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    t = 0
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
        t = t + 1
        t0 = time.time()
        _train_loss, _train_accuracy, _, _,_ = sess.run([train_loss, train_accuracy, update_Ws, update_grads_moment, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_adam, qs_r_kron_adam = sess.run([Qs_left, Qs_right])

    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))

  
scipy.io.savemat(results_dir + 'moment_kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.406951665878296; test loss: 2.3741438388824463; train accuracy: 0.0390625; test accuracy: 0.06459999829530716; Time: 2.4746944904327393
100: train loss: 1.089107416638732; test loss: 0.21023276448249817; train accuracy: 0.65240234375; test accuracy: 0.9447000026702881; Time: 4.005738973617554
200: train loss: 0.5471953644593693; test loss: 0.08581697940826416; train accuracy: 0.8313060003442991; test accuracy: 0.9758999943733215; Time: 5.526403903961182
300: train loss: 0.28972674048449093; test loss: 0.05715553089976311; train accuracy: 0.9127552949547308; test accuracy: 0.9843999743461609; Time: 7.045264959335327
400: train loss: 0.16903055083515342; test loss: 0.04379257187247276; train accuracy: 0.9495822703673328; test accuracy: 0.9884999990463257; Time: 8.56517481803894
500: train loss: 0.1160991962011241; test loss: 0.03813538700342178; train accuracy: 0.9663241130682976; test accuracy: 0.989300012588501; Time: 10.083621740341187
600: train loss: 0.0895030199898

## Kronecker Product --> ADAM

In [19]:
# using grads to compute preconditioner, then applying adam on precond_grads
# grads ---> precond_grads ---> adam_grads

step_size = 0.02
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   
beta1 = 0.9
beta2 = 0.999
offset = 1e-9

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    grads_vars = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]
    grads_moment = [tf.Variable(tf.zeros(W.shape, dtype=dtype), trainable=False) for W in Ws]

    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    Qs_right = [tf.Variable(tf.eye(W.shape.as_list()[1], dtype=dtype), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)

    precond_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, grads)]

    new_grads_moment = [beta1*old + (1.0 - beta1)*new for (old, new) in zip(grads_moment, precond_grads)]
    new_grads_vars = [beta2*old + (1.0 - beta2)*new*new for (old, new) in zip(grads_vars, precond_grads)]
    new_grads_moment_hat = [m/(1-beta1**adam_step) for m in new_grads_moment]
    new_grads_vars_hat = [v/(1-beta2**adam_step) for v in new_grads_vars]

    adam_grads = [m/tf.sqrt(v + offset) for (m,v) in zip(new_grads_moment_hat,new_grads_vars_hat)]

    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in adam_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    
    new_Ws = [W - (step_size_adjust*step_size)*pG for (W, pG) in zip(Ws,adam_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    update_grads_vars = [tf.assign(old, new) for (old, new) in zip(grads_vars, new_grads_vars)]
    update_grads_moment = [tf.assign(old, new) for (old, new) in zip(grads_moment, new_grads_moment)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    
    test_loss, test_accuracy = test_criterion(Ws)  
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    t = 0
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
        t = t + 1
        t0 = time.time()
        _train_loss, _train_accuracy, _, _,_,_ = sess.run([train_loss, train_accuracy, update_Ws, update_grads_moment, update_grads_vars, update_Qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs, adam_step: t})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_adam, qs_r_kron_adam = sess.run([Qs_left, Qs_right])

    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))
  
scipy.io.savemat(results_dir + 'kron_adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3989572525024414; test loss: 2.201631784439087; train accuracy: 0.03125; test accuracy: 0.20160000026226044; Time: 2.9666478633880615
100: train loss: 0.5689112619027499; test loss: 0.06846974790096283; train accuracy: 0.8184710937500004; test accuracy: 0.9786999821662903; Time: 4.533591032028198
200: train loss: 0.27734027824759167; test loss: 0.041201137006282806; train accuracy: 0.9118718769629609; test accuracy: 0.9873999953269958; Time: 6.088185548782349
300: train loss: 0.1422005983467293; test loss: 0.040101733058691025; train accuracy: 0.9550320690532619; test accuracy: 0.9873999953269958; Time: 7.639142751693726
400: train loss: 0.08607102500470497; test loss: 0.03758822754025459; train accuracy: 0.9734741435426392; test accuracy: 0.9876999855041504; Time: 9.197100639343262
500: train loss: 0.0670070886840364; test loss: 0.035305242985486984; train accuracy: 0.9795461754261564; test accuracy: 0.9882000088691711; Time: 10.751317024230957
600: train loss: 0.0545

# KRON & SCAN 

## Scan + Kron AVERAGE

In [20]:
# grads ---> kron_grads ---> scan_grads

step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    Qs_right = [tf.Variable(tf.eye(W.shape.as_list()[1], dtype=dtype), trainable=False) for W in Ws]
    qs_left = [tf.Variable(tf.concat([tf.ones((1, W.shape.as_list()[0])),tf.zeros((1, W.shape.as_list()[0]))], axis=0), trainable=False) for W in Ws]
    qs_right = [tf.Variable(tf.ones((1, W.shape.as_list()[1])), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)

    scan_grads = [psgd.precond_grad_scan(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    kron_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, grads)]
    precond_grads = [(s+k)/2 for (s,k) in zip(scan_grads, kron_grads)]

    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in precond_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    
    new_Ws = [W - (step_size_adjust*step_size)*pG for (W, pG) in zip(Ws, precond_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    new_qs = [psgd.update_precond_scan(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
  
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
        
        t0 = time.time()
        _train_loss, _train_accuracy, _, _,_ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_adam, qs_r_kron_adam = sess.run([Qs_left, Qs_right])

    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))
  
scipy.io.savemat(results_dir + 'scan_kron_avg.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.39605712890625; test loss: 2.278923273086548; train accuracy: 0.0703125; test accuracy: 0.13449999690055847; Time: 3.580127239227295
100: train loss: 0.9646626208752392; test loss: 0.1982114017009735; train accuracy: 0.7088625; test accuracy: 0.9476000070571899; Time: 5.319075345993042
200: train loss: 0.51470116346077; test loss: 0.09686628729104996; train accuracy: 0.8433943271644563; test accuracy: 0.9732000231742859; Time: 7.038480520248413
300: train loss: 0.2836559792185827; test loss: 0.07133886218070984; train accuracy: 0.9152173601127125; test accuracy: 0.978600025177002; Time: 8.764195680618286
400: train loss: 0.18007598716941076; test loss: 0.04824352264404297; train accuracy: 0.9451301947411288; test accuracy: 0.9855999946594238; Time: 10.489600896835327
500: train loss: 0.12124288467226452; test loss: 0.04143662005662918; train accuracy: 0.9628255803023215; test accuracy: 0.9866999983787537; Time: 12.214603900909424
600: train loss: 0.0989577408634995; te

## Scan -> Kron

In [21]:
# grads ---> kron_grads ---> scan_grads

step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   
beta1 = 0.9
beta2 = 0.999
offset = 1e-9

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    Qs_right = [tf.Variable(tf.eye(W.shape.as_list()[1], dtype=dtype), trainable=False) for W in Ws]
    qs_left = [tf.Variable(tf.concat([tf.ones((1, W.shape.as_list()[0])),tf.zeros((1, W.shape.as_list()[0]))], axis=0), trainable=False) for W in Ws]
    qs_right = [tf.Variable(tf.ones((1, W.shape.as_list()[1])), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)

    scan_grads = [psgd.precond_grad_scan(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, grads)]
    kron_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, scan_grads)]
    

    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in kron_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    
    new_Ws = [W - (step_size_adjust*step_size)*pG for (W, pG) in zip(Ws, kron_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    new_qs = [psgd.update_precond_scan(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    
    test_loss, test_accuracy = test_criterion(Ws)   
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    t = 0
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
        
        t0 = time.time()
        _train_loss, _train_accuracy, _, _,_ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_adam, qs_r_kron_adam = sess.run([Qs_left, Qs_right])

    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))
  
scipy.io.savemat(results_dir + 'scan_kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3781280517578125; test loss: 2.277127265930176; train accuracy: 0.0859375; test accuracy: 0.1200999990105629; Time: 4.070016860961914
100: train loss: 0.7676858145534993; test loss: 0.09397528320550919; train accuracy: 0.7621632812499998; test accuracy: 0.9725000262260437; Time: 5.792638778686523
200: train loss: 0.36941787397162573; test loss: 0.05511372163891792; train accuracy: 0.8865091566246319; test accuracy: 0.984499990940094; Time: 7.506967306137085
300: train loss: 0.19741662952486794; test loss: 0.04214391112327576; train accuracy: 0.9401871865058342; test accuracy: 0.9876999855041504; Time: 9.223902940750122
400: train loss: 0.11716683577549737; test loss: 0.034236837178468704; train accuracy: 0.9656504383839707; test accuracy: 0.9894999861717224; Time: 10.942944765090942
500: train loss: 0.08279337729603953; test loss: 0.031033361330628395; train accuracy: 0.9757425624348106; test accuracy: 0.9908999800682068; Time: 12.666457891464233
600: train loss: 0.065

## Kron -> Scan

In [22]:
# grads ---> kron_grads ---> scan_grads

step_size = 0.05
grad_norm_clip_thr = 10   # gradients clipping may be necessary for RNN training; 
                            # set it to an extremely large value if no clipping is required   
beta1 = 0.9
beta2 = 0.999
offset = 1e-9

with tf.Session() as sess:   
    eps = max([np.finfo(W.dtype.as_numpy_dtype).eps for W in Ws])
    
    Qs_left = [tf.Variable(tf.eye(W.shape.as_list()[0], dtype=dtype), trainable=False) for W in Ws]
    Qs_right = [tf.Variable(tf.eye(W.shape.as_list()[1], dtype=dtype), trainable=False) for W in Ws]
    qs_left = [tf.Variable(tf.concat([tf.ones((1, W.shape.as_list()[0])),tf.zeros((1, W.shape.as_list()[0]))], axis=0), trainable=False) for W in Ws]
    qs_right = [tf.Variable(tf.ones((1, W.shape.as_list()[1])), trainable=False) for W in Ws]
    
    train_loss, train_accuracy = train_criterion(Ws)
    grads = tf.gradients(train_loss, Ws)

    kron_grads = [psgd.precond_grad_kron(ql, qr, g) for (ql, qr, g) in zip(Qs_left, Qs_right, grads)]
    scan_grads = [psgd.precond_grad_scan(ql, qr, g) for (ql, qr, g) in zip(qs_left, qs_right, kron_grads)]

    grad_norm = tf.sqrt(tf.reduce_sum([tf.reduce_sum(g*g) for g in scan_grads]))
    step_size_adjust = tf.minimum(1.0, grad_norm_clip_thr/(grad_norm + 1.2e-38))
    
    new_Ws = [W - (step_size_adjust*step_size)*pG for (W, pG) in zip(Ws, scan_grads)]
    update_Ws = [tf.assign(W, new_W) for (W, new_W) in zip(Ws, new_Ws)]
    
    delta_Ws = [tf.random_normal(W.shape, dtype=dtype) for W in Ws]
    grad_deltaw = tf.reduce_sum([tf.reduce_sum(g*v) for (g, v) in zip(grads, delta_Ws)])
    hess_deltaw = tf.gradients(grad_deltaw, Ws)
    
    new_Qs = [psgd.update_precond_kron(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(Qs_left, Qs_right, delta_Ws, hess_deltaw)]
    update_Qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(Qs_left, Qs_right, new_Qs)]
    new_qs = [psgd.update_precond_scan(ql, qr, dw, dg) for (ql, qr, dw, dg) in zip(qs_left, qs_right, delta_Ws, hess_deltaw)]
    update_qs = [[tf.assign(old_ql, new_q[0]), tf.assign(old_qr, new_q[1])] for (old_ql, old_qr, new_q) in zip(qs_left, qs_right, new_qs)]
    
    
    test_loss, test_accuracy = test_criterion(Ws)
    
    sess.run(tf.global_variables_initializer())
    avg_train_loss = 0.0
    avg_train_acc= 0.0
    TrainLoss = list()
    TestLoss = list()
    TrainAccuracy = list()
    TestAccuracy = list()
    Time = list()
    
    for num_iter in range(ITERATIONS):    
        _train_inputs, _train_outputs = get_batches( )
      
        t0 = time.time()
        _train_loss, _train_accuracy,  _, _,_ = sess.run([train_loss, train_accuracy, update_Ws, update_Qs, update_qs],
                                     {train_inputs: _train_inputs, train_outputs: _train_outputs})
        Time.append(time.time() - t0)
        nu = min(num_iter/(1.0 + num_iter), 0.99)
        avg_train_loss = nu*avg_train_loss + (1.0 - nu)*_train_loss
        avg_train_acc = nu*avg_train_acc + (1.0 - nu)*_train_accuracy
        TrainLoss.append(avg_train_loss)
        TrainAccuracy.append(avg_train_acc)
        if num_iter % GAP == 0:
            _test_loss, _test_accuracy = sess.run([test_loss, test_accuracy])
            TestLoss.append(_test_loss)
            TestAccuracy.append(_test_accuracy)
            print('{}: train loss: {}; test loss: {}; train accuracy: {}; test accuracy: {}; Time: {}'.format(
                num_iter, TrainLoss[-1], TestLoss[-1], TrainAccuracy[-1], TestAccuracy[-1], np.sum(Time)))
        if num_iter % 19999 == 0:
           qs_l_kron_adam, qs_r_kron_adam = sess.run([Qs_left, Qs_right])

    t = final_outputs(Ws)
    test_predictions = sess.run(t)

# determine precision and recall
y_ = np.asarray(test_predictions)
y_ = np.reshape(y_, (10000,))
print(metrics.confusion_matrix(test_outputs, y_))
print(metrics.classification_report(test_outputs, y_, digits=4))
  
scipy.io.savemat(results_dir + 'kron_scan.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'TrainAccuracy': TrainAccuracy,'TestAccuracy': TestAccuracy, 'Time':Time})

0: train loss: 2.3923301696777344; test loss: 2.274977445602417; train accuracy: 0.078125; test accuracy: 0.11710000038146973; Time: 4.593583106994629
100: train loss: 0.7739226437717679; test loss: 0.12078545987606049; train accuracy: 0.7606890625000002; test accuracy: 0.9631999731063843; Time: 6.326172351837158
200: train loss: 0.3625540239915501; test loss: 0.04556639492511749; train accuracy: 0.8884151881095436; test accuracy: 0.9861999750137329; Time: 8.035696268081665
300: train loss: 0.1766458531688637; test loss: 0.0329323410987854; train accuracy: 0.9455331704944209; test accuracy: 0.989300012588501; Time: 9.752628803253174
400: train loss: 0.10188747403483543; test loss: 0.03168304264545441; train accuracy: 0.9690290092832704; test accuracy: 0.9901999831199646; Time: 11.467976331710815
500: train loss: 0.06749131294550925; test loss: 0.027846014127135277; train accuracy: 0.9794810425534345; test accuracy: 0.9911999702453613; Time: 13.184107065200806
600: train loss: 0.0489623

# Comparison of Results

In [0]:
opts = ['SGD','adam','rmsprop','kron_psgd','scan_psgd','scaw_psgd', 'reverse_scan_psgd','reverse_scaw_psgd',
        'scan_pro_psgd','hybrid_skron_psgd','hybrid_kscan_psgd','kdiag_psgd', 'kron_adam','adam_kron',
				'kron_scan','scan_kron','scan_kron_avg','moment_kron','test']

opts = ['SGD','adam','kron_psgd','hybrid_kscan_psgd','scan_psgd','kron_kron','kron_scan']

total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	
  

In [0]:
for opt in opts:
  # print(opt)
  data = opts_data[opt]
  times[opt] = data.get('Time')
  train_times[opt] = np.cumsum(times[opt])
  total_train_time[opt] = np.sum(times[opt])
  train_losses[opt] = data.get('TrainLoss').reshape(ITERATIONS,)
  train_accs[opt] = data.get('TrainAccuracy').reshape(ITERATIONS,)
  test_losses[opt] = data.get('TestLoss').reshape(int(ITERATIONS/GAP),)
  test_accs[opt] = data.get('TestAccuracy').reshape(int(ITERATIONS/GAP,))
  tt = []
  for i in range(0,ITERATIONS,GAP):
    tt.append(train_times[opt][i])
  test_times[opt] = tt

In [0]:
# colors = ['b','g','r','c','m','y','k','orange','lime','aqua','deeppink','darkviolet','chocolate','midnightblue','coral'] ,matplotlib
# colors = px.colors.qualitative.Dark24_r
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#83FF33','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [0]:

# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs Iterations', 'Iterations','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs Iterations', 'Iterations','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')

In [0]:
# plot train_losses vs Iterations
plot_acc_metrics(None,train_accs,'Train Accuracy vs Iterations', 'Iterations','Train Accuracy')
# plot test_losses vs Iterations
plot_acc_metrics(None,test_accs,'Test Accuracys vs Iterations', 'Iterations','Test Accuracy')
# # plot test_losses vs Iterations
plot_acc_metrics(train_times,train_accs,'Train Accuracy vs Time', 'Time','Train Accuracy')
# plot test_losses vs Iterations
plot_acc_metrics(test_times,test_accs,'Test Accuracy vs Time', 'Time','Test Accuracy')

In [0]:
 print(tabulate(zip([(k) for k,v in total_train_time.items()],
                   [v for k,v in total_train_time.items()],
                   [v/ITERATIONS for k,v in total_train_time.items()]),headers = ['optimizers','total time (sec)','time per iteration']))

**A. Loss vs Epochs**


1.   Training:  KRON_KRON is best. Though At the start, all combinations converge better than simple KRONECKER preconditioner. But later, above mentioned seems to work best while SCAN_KRON and KRON_ADAM seems to work worst. 
Although KRON_SCAN works comparable even in latter stages. Not too bad 
2.   Testing: SCAN_KRON_AVG, KRON_KRON which work best in training start to overfit. KRON_SCAN is the best one. And SCAN_KRON works comparable to it

SCAN.KRON and its sqrt doesn't seem to work
**B. Accuracy vs Epochs**
KRON_SCAN is best in testing and training. ADAM_KRON overfits though good in training

CONCLUSION: KRON_SCAN is the winner


# NGROK LAUNCH TENSORBOARD

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
import plotly.graph_objects as go
import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=x, y=x**2))
fig.show()